In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from pprint import pprint

os.environ['LANGCHAIN_PROJECT']
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY') # Not required to call this code explicitly!

## ChatOpenAI

In [3]:
# Initialise the model
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-5-nano-2025-08-07')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001FCE4363E60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001FCFF31B710>, root_client=<openai.OpenAI object at 0x000001FCFF31AF60>, root_async_client=<openai.AsyncOpenAI object at 0x000001FCFF31A870>, model_name='gpt-5-nano-2025-08-07', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [ ]:
# Invoke/Make a call from the model
res1 = llm.invoke("What is Chelsea FC?")
res1

AIMessage(content='Chelsea FC (Chelsea Football Club) is a professional soccer club based in London, England. Here are the basics:\n\n- Based in: Fulham, London\n- Stadium: Stamford Bridge\n- Founded: 1905\n- Competition: Premier League (the top level of English football)\n- Colors/Nickname: Royal blue; nicknamed “The Blues” (and historically “The Pensioners”)\n- Notable achievements: one of England’s most successful clubs in the 21st century, with multiple domestic trophies (Premier League titles, FA Cups, League Cups) and major European honours (two UEFA Champions League titles and two UEFA Europa League titles)\n- Ownership: owned by a consortium led by Todd Boehly and Clearlake Capital (as of the latest major changes)\n\nIf you’d like, I can give a brief history, talk about famous players, or cover their most memorable moments.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2175, 'prompt_tokens': 11, 'total_tokens': 2186, 'completion_

In [22]:
# Get the proper output format
pprint(res1.content)

('Chelsea FC (Chelsea Football Club) is a professional soccer club based in '
 'London, England. Here are the basics:\n'
 '\n'
 '- Based in: Fulham, London\n'
 '- Stadium: Stamford Bridge\n'
 '- Founded: 1905\n'
 '- Competition: Premier League (the top level of English football)\n'
 '- Colors/Nickname: Royal blue; nicknamed “The Blues” (and historically “The '
 'Pensioners”)\n'
 '- Notable achievements: one of England’s most successful clubs in the 21st '
 'century, with multiple domestic trophies (Premier League titles, FA Cups, '
 'League Cups) and major European honours (two UEFA Champions League titles '
 'and two UEFA Europa League titles)\n'
 '- Ownership: owned by a consortium led by Todd Boehly and Clearlake Capital '
 '(as of the latest major changes)\n'
 '\n'
 'If you’d like, I can give a brief history, talk about famous players, or '
 'cover their most memorable moments.')


## ChatPromptTemplate

In [ ]:
# Creating a prompt Template
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert AI Engineer. Answer the questions in crisp and easy to understand manner"),
        ("user", "{input}")
    ])
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Answer the questions in crisp and easy to understand manner'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [23]:
chain = prompt|llm

res2 = chain.invoke({"input":"What are retrievers and vector databases?"})

pprint(res2.content)

('Here’s a concise way to think about them and how they fit together.\n'
 '\n'
 'What is a retriever?\n'
 '- A component that, given a user query, fetches relevant documents or '
 'passages from a large collection.\n'
 '- It usually works by comparing the query to document representations '
 '(embeddings) or using keyword matches.\n'
 '- Types:\n'
 '  - Dense retrievers: use neural embeddings (high-dimensional vectors) to '
 'measure similarity.\n'
 '  - Sparse retrievers: rely on keyword-based methods like BM25.\n'
 '  - Hybrid retrievers: combine dense and sparse signals.\n'
 '\n'
 'What is a vector database?\n'
 '- A specialized database that stores vector representations (embeddings) of '
 'many items (documents, passages, etc.).\n'
 '- It lets you perform fast similarity searches (nearest-neighbor search) to '
 'find items whose vectors are closest to a query vector.\n'
 '- Features often include: metadata filtering, hybrid search (combine vector '
 'similarity with keyword filter

## OutputParser

In [24]:
from langchain_core.output_parsers import StrOutputParser
outputParser = StrOutputParser()

chain = prompt|llm|outputParser

pprint(chain.invoke({"What are tensors?"}))

('- A tensor is a multi-dimensional array of numbers. It’s the generalization '
 'of scalars, vectors, and matrices.\n'
 '- Rank (or order):\n'
 '  - 0D: scalar (e.g., 3.14)\n'
 '  - 1D: vector (e.g., [1, 2, 3])\n'
 '  - 2D: matrix (e.g., 3x4)\n'
 '  - 3D+: higher-dimensional arrays (e.g., a stack of images, or a batch of '
 'data)\n'
 '- Shape: describes the size along each dimension. Examples:\n'
 '  - scalar: ()\n'
 '  - vector of length 5: (5,)\n'
 '  - matrix 2x3: (2, 3)\n'
 '  - image 64x64 RGB: (64, 64, 3)\n'
 '  - batch of 32 images: (32, 64, 64, 3)\n'
 '- Key properties: shape, data type (dtype), and device (CPU/GPU).\n'
 '- Why it matters in ML: tensors are the primary data containers; they store '
 'inputs, parameters, and outputs, support a wide range of operations '
 '(element-wise math, matmul, reductions), and can be tracked for automatic '
 'differentiation.\n'
 '\n'
 'In short: a tensor is a flexible, multi-dimensional data container used to '
 'represent and compute w

## Simple App#1

In [4]:
from langchain_community.document_loaders import WebBaseLoader

url = "https://docs.smith.langchain.com/tutorials/Administrators/manage_spend"

wb_loader = WebBaseLoader(url)
scraped_data = wb_loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter
rct_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 20)
splitted_data = rct_splitter.split_documents(scraped_data)

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(splitted_data,embeddings)

## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt)

from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

retriever=vectorstoredb.as_retriever()

from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']


'The LangSmith docs have moved to the LangChain Docs site. You can find the LangSmith documentation there.'